In [ ]:
import numpy as np
import numpy as np
from scipy import signal
from scipy import datasets

def single_scan(neur_vol = None,psf_sz = None,psf2 = None,varargin = None): 
    # scan_img = scan_volume(neur_vol, psf, varargin)
    
    # Scan a 3D volume with a given point-spread function. This function takes
# in a neural volume (as generated by "simulate_neural_vol_v*.m"), a
# point-spread function, and a number of other optional scanning parameters
# in order to simulate the images resulting from a two-photon scan of the
# volume with the given PSF. Inputs to this function are:
#   - neur_vol - 3D volume where each voxel contains the fluorescence
#   - psf      - 3D array containing the intensity of the point-spread
#                function
#   - z_sub    - OPTIONAL speed-up parameter that scans multiple slices
#                simultaneously, reducing the number of convolutions needed
#                by a factor of 'z_sub'
    
    # The output is
#   - scan_img - The scanned image - i.e. the return fluorescence from the
#                point-spread function with no noise (photon or electronic)
    
    # 2016 - Adam Charles
    
    ###########################################################################
## Parse Inputs
    
    if len(varargin) > 3:
        z_sub = varargin[0]
    else:
        z_sub = 1
    
    if len(varargin) > 4:
        freq_opt = varargin[2]
    else:
        freq_opt = False
    
    ###########################################################################
## Calculate and sum convolutions
    
    if z_sub > 1:
        N_slce = np.ceil(neur_vol.shape[3-1] / z_sub)
        neur_vol2 = neur_vol[:,:,np.arange(1,z_sub * N_slce+z_sub,z_sub)]
        if not freq_opt :
            psf = psf2(:,:,np.arange(1,z_sub * N_slce+z_sub,z_sub))
        for kk in np.arange(2,z_sub+1).reshape(-1):
            slcs = np.arange(kk,np.amin(z_sub * N_slce,neur_vol.shape[3-1])+z_sub,z_sub)
            Nz = np.asarray(slcs).size
            neur_vol2[:,:,np.arange[1,Nz+1]] = neur_vol2[:,:,np.arange(1,Nz+1)] + neur_vol[:,:,slcs]
            if not freq_opt :
                psf[:,:,np.arange[1,Nz+1]] = psf[:,:,np.arange(1,Nz+1)] + psf2[:,:,slcs]
    
    if freq_opt:
        sz = psf2.shape
        scan_img = np.ifft(np.ifft(np.sum(np.multiply(np.fft.rfft(np.fft.rfft(neur_vol2,sz[1],1),sz[2],2),psf2), 3-1),[],1),[],2,'symmetric')
        # sz = size(neur_vol2) + size(psf) - 1;                                     # Get the sizes of the post-convolution array
# scan_img = ifft(ifft(fft(fft(neur_vol2, sz(1), 1), sz(2), 2) .* ...
#           fft(fft(psf2, sz(1), 1), sz(2), 2), [], 1), [], 2, 'symmetric'); # Perform convolution in the Fourier domain in both NON-axial dimensions
#     scan_img = sum(scan_img, 3);                                           # Sum along the axial dimension
        y_ix = np.ceil((psf_sz(1) - 1) / 2) + np.array([1,neur_vol2.shape[1-1]])
        y_jx = np.ceil((psf_sz(2) - 1) / 2) + np.array([1,neur_vol2.shape[2-1]])
        scan_img = scan_img(np.arange(y_ix(1),y_ix(2)+1),np.arange(y_jx(1),y_jx(2)+1))
    else:
        scan_img = 0
        for ll in np.arange(1,psf2.shape[3-1]+1).reshape(-1):
            scan_img = scan_img + signal.convolve2d(neur_vol[:,:,ll],psf2[:,:,ll],'same')
    
    ###########################################################################
###########################################################################
    return scan_img